In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
HOME = os.getcwd()
print(HOME)
ANIME_DATA = os.path.join(HOME, r'Preprocessing\Preprocessed Data\raw_anime_processed.csv')
print(ANIME_DATA)

d:\Data Science Introduction\Final_Project\models\anime_recommedation_system
d:\Data Science Introduction\Final_Project\models\anime_recommedation_system\Preprocessing\Preprocessed Data\raw_anime_processed.csv


In [3]:
df = pd.read_csv(ANIME_DATA)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Data Science Introduction\\Final_Project\\models\\anime_recommedation_system\\Preprocessing\\Preprocessed Data\\raw_anime_processed.csv'

Attributes  dataset

In [ ]:
df.keys()

Index(['Title', 'Score', 'Vote', 'Ranked', 'Popularity', 'Members', 'Favorite',
       'Types', 'Volumes', 'Chapters', 'Status', 'Published', 'Genres',
       'Themes', 'Demographic', 'Serialization', 'Author', 'Total Review',
       'Type Review', 'Realeased date', 'Completed date', 'Recommended',
       'Mixed Feelings', 'Not Recommended'],
      dtype='object')

Filling empty values

In [ ]:
df['Genres'] = df['Genres'].fillna(str(['''''']))
df['Themes'] = df['Themes'].fillna(str(['''''']))

Normalize string attributes

In [ ]:
# Chuyển các thuộc tính dạng list thành chuỗi văn bản
df['Genres_str'] = df['Genres'].apply(lambda x: " ".join([genre.strip("'") for genre in x.split(',')]) if x and isinstance(x, str) else "")
df['Themes_str'] = df['Themes'].apply(lambda x: " ".join([theme.strip("'") for theme in x.split(',')]) if x and isinstance(x, str) else "")
df['Author_str'] = df['Author'].apply(lambda x: " ".join([author.strip("'") for author in x.split(',')]) if x and isinstance(x, str) else "")

# Áp dụng TF-IDF cho các thuộc tính dạng chuỗi
tfidf = TfidfVectorizer(stop_words='english')

tfidf_Title_matrix = tfidf.fit_transform(df['Title'])
tfidf_Types_matrix = tfidf.fit_transform(df['Types'])
tfidf_Genres_matrix = tfidf.fit_transform(df['Genres'])
tfidf_Themes_matrix = tfidf.fit_transform(df['Themes'])
tfidf_Author_matrix = tfidf.fit_transform(df['Author'])

Normalize number attributes

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Chuẩn hóa các thuộc tính dạng số
scaler = MinMaxScaler()
df[['Score', 'Vote', 'Ranked', 'Popularity', 'Recommended']] = scaler.fit_transform(df[['Score', 'Vote', 'Ranked', 'Popularity', 'Recommended']])


In [ ]:
X_tfidf = sp.hstack([
    tfidf_Title_matrix,       # Giữ dạng sparse
    tfidf_Types_matrix,
    tfidf_Genres_matrix,
    tfidf_Themes_matrix,
    tfidf_Author_matrix,
    sp.csr_matrix(df[['Score']].values),  # Chuyển các cột số sang sparse
    sp.csr_matrix(df[['Vote']].values),
    sp.csr_matrix(df[['Ranked']].values),
    sp.csr_matrix(df[['Popularity']].values),
    sp.csr_matrix(df[['Recommended']].values)
])


In [ ]:
# Tính ma trận độ tương đồng cosine
cosine_sim = cosine_similarity(X_tfidf, X_tfidf)

Attributes selection using PCA

In [ ]:
def recommend_anime(title, cosine_sim, df, top_n=5):
    # Step 1: Find the index (anime_id) of the anime matching the title
    idx = df[df['Title'] == title].index

    if idx.empty:
        print(f"No anime found with title: {title}")
        return []
    
    idx = idx[0]  # Get the first match (assuming titles are unique)

    # Step 2: Get the similarity scores for this anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Step 3: Sort by similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Step 4: Get the top_n most similar anime (excluding itself)
    sim_scores = sim_scores[1:top_n+1]  # Skip the first, as it’s the same anime

    # Step 5: Retrieve the anime titles for recommendations
    anime_indices = [i[0] for i in sim_scores]
    recommended = df.iloc[anime_indices]

    return recommended[['Title', 'Score']]  # Return relevant columns


In [ ]:
title_to_search = "One Piece"
recommendations = recommend_anime(title_to_search, cosine_sim, df, top_n=5)

if not recommendations.empty:
    print("Recommended anime:")
    print(recommendations)
else:
    print("No recommendations found.")


Recommended anime:
                                                Title     Score
599     One Piece: Episode A (One Piece: Ace's Story)  0.803977
5029                   One Piece: Mugiwara Daigekijou  0.688920
4637  Wanted! (Wanted! Eiichiro Oda Before One Piece)  0.693182
628                           One Piece: Strong World  0.801136
46                                    Hunter x Hunter  0.897727
